In [ ]:
import torch; # 引入torch库
from torch import nn; # 从torch库中引入神经网络模块
from torch.utils.data import DataLoader, Dataset; # 引入数据加载和数据集模块
from torchvision import transforms; # 引入图像转换模块
from torchvision.transforms import ToTensor; # 引入将图像转换为张量的模块
